# Interpreting what aspects of a DnD 5e characters indicate that they are multiclassing.

## Abstract:


## Design:
I am interested in building an interpretive classification model that utilizes DnD character sheet data to determine whether a character is multiclassing. The entire goal of this project is to ascertain which features indicate class and how their interactions influence their class.

## Data
All character sheet data was obtained from GitHub user [oganm's](https://github.com/oganm) repository [dnddata](https://github.com/oganm/dnddata). It is a weekly updated dataset of characters that are submitted to his web applications [printSheetApp](https://oganm.com/shiny/printSheetApp/) and [interactiveSheet](https://oganm.com/shiny/interactiveSheet/). He did some exploratory data analysis (EDA) on the dataset to determine rarity of DnD characters in a [dndstats blogpost](https://oganm.github.io/dndstats/). This was based on an article by FiveThirtyEight titled: [Is Your D&D Character Rare?](https://fivethirtyeight.com/features/is-your-dd-character-rare/)

The [**dataset**](https://github.com/oganm/dnddata/blob/master/data-raw/dnd_chars_all.tsv) has 9784 character sheets, with about 19 features. Some of them Oganm processed which was nice. Many had to be one-hot encoded and categorized to feed into the model.

## Algorithms:
*Feature Engineering*

The classification model algorithms for this project that will be tested are: **Logistic Regression, Random Forest, and Naive Bayes (Bernoulli and Guassian)**. Classification model evaluation metric focus is f-1 score.
- Still analyze results with Precision, Recall, Confusion Matrix, and ROC Curves

## Tools:
- sklearn, pandas, numpy, and matplotlib
- Tableau and seaborn for visualization
    - To see the interactive dashboard and EDA for the dataset click [here](https://public.tableau.com/app/profile/louisa.reilly/viz/dnd5e_char/DnD_Char?publish=yes).

### Sources/Acknowledgements:
- Gary Gygax and Dave Arneson for creating DnD.
- Wizards of the Coast(subsidary of Hasbro) which publishes DnD guides.
- B. Ogan Mancarci for his data collection. [GitRepo](https://github.com/oganm/dnddata)
- Dan Quach for his 2020 [blogpost](https://towardsdatascience.com/classifying-character-classes-in-dungeons-dragons-with-machine-learning-86751240594d) about creating a classifier using Oganm's dataset.